# 네이버 매장 -> 매장에 있는 리뷰 스크래핑

문제
 
1. 추천순으로 긁어올지, 최신순으로 긁어올지

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

import datetime
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd

c:\Users\hare0\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
url = 'https://m.place.naver.com/restaurant/13149660/review/visitor?entry=pll'

driver = webdriver.Chrome(ChromeDriverManager().install())
res = driver.get(url)
driver.implicitly_wait(10)

# # 최신순 리뷰
# driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
# time.sleep(0.4)

# 추천순 리뷰
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[1]/a').click()
time.sleep(0.4)


# html = driver.page_source
# #soup = BeautifulSoup(html, 'lxml')
# soup = BeautifulSoup(driver.page_source, 'html.parser')


In [3]:
# tag - 이런점이 좋았어요 가져오기 
# tag 끝까지 나오게 클릭 
try :
    while driver.find_elements(By.CLASS_NAME, 'Tvx37'):
        try:
            button = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/a[1]')
            button.click()
            time.sleep(0.4)
        except ElementClickInterceptedException:
            break
except NoSuchElementException:
    print("No more '더보기' button found. Exiting the loop.")

# tag 불러오기 
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')
items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

# 'tag'와 '인원'을 추출하여 새로운 데이터프레임 생성
data = {
    'tag': [item.text.split('"')[1] for item in items],
    '인원': [int(''.join(filter(str.isdigit, item.text))) for item in items]
}

df = pd.DataFrame(data)

# 결과 출력
df

,tag,인원
0,음식이 맛있어요,1131
1,재료가 신선해요,735
2,혼밥하기 좋아요,618
3,친절해요,301
4,가성비가 좋아요,257
5,매장이 청결해요,241
6,매장이 넓어요,208
7,특별한 메뉴가 있어요,130
8,양이 많아요,125
9,인테리어가 멋져요,42


In [72]:
# 매장명
place = '강남역_서브웨이'

# 엑셀파일 만들기 
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet(place)
list_sheet.append(['nickname', 'content', 'date', 'revisit','tag'])

# 리뷰 스크래핑 시작
try:
    # driver = webdriver.Chrome(ChromeDriverManager().install())
    # res = driver.get(url)
    # driver.implicitly_wait(2)

    for _ in range(5): # 리뷰 더보기 5번만...
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.4)

        # Pagedown
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

        try:
            while True:
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
                time.sleep(0.4)
        except Exception as e:
            print('finish')

        time.sleep(30)
        html = driver.page_source
        bs = BeautifulSoup(html, 'lxml')
        reviews = bs.select('li.YeINN')

        for r in reviews:
            nickname = r.select_one('div.VYGLG')
            content = r.select_one('div.ZZ4OK.IwhtZ')
            tag = r.select_one('div.gyAGI span.P1zUJ').text
            date = r.select('div._7kR3e>span.tzZTd>time')[0]
            revisit = r.select('div._7kR3e>span.tzZTd')[1]

            # 아무것도 없을 때
            nickname = nickname.text if nickname else ''
            content = content.text if content else ''
            tag = tag if tag else ''
            date = date.text if date else ''
            revisit = revisit.text if revisit else ''
            time.sleep(0.06)

            print(nickname, '/', content, '/', date, '/', revisit, '/',tag)
            list_sheet.append([nickname, content, date, revisit,tag])
            time.sleep(0.06)

        # 저장하기
        file_name = 'naver_review_' + place + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
        xlsx.save(file_name)

# 에러가 나는 경우
except Exception as e:
    print(e)
    # Save the file(temp)
    file_name = 'naver_review_'+ place + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

finish
맛집학과 수석 / 신메뉴 랍스터 나왔다해서 먹으러 왔어요!! 가격이 좀 비싸긴 했지만 랍스터 첫입 먹을 때 그 랍스터 향기는 잊을 수가 없네요!! 진짜 맛있구 또 먹을거에요!! :) / 23.12.11.월 / 8번째 방문 / 음식이 맛있어요
Favor10 / 키오스크가 있어서 편해요!! / 2.4.일 / 5번째 방문 / 음식이 맛있어요
토랑꾸 / 사람이 많은데 빨리 먹기 괜찮고, 맛있어요!
행사해서 다른 서브웨이보다 가격도 저렴하고 좋아요😀 / 1.22.월 / 1번째 방문 / 가성비가 좋아요
NancyK /  / 23.12.14.목 / 2번째 방문 / 음식이 맛있어요
쪼꼬미쪼콩 / 빠르고 맛있게 만들어줘요!👍 / 1.22.월 / 2번째 방문 / 음식이 맛있어요
별땅20 / 랍스터가 많이 들어있네요. 가격이 많이 비싸지만 맛있게 먹었네요.  / 23.12.26.화 / 1번째 방문 / 음식이 맛있어요
맛집학과 수석 / 진짜 여기 섭웨가 잘 만드시는 것 같아요! 로스팅이 항상 잘 되어나와서 바삭한 빵을 만날 수 있어요 :) / 23.11.30.목 / 7번째 방문 / 혼밥하기 좋아요
dldkdud2150 / 첫 서브웨이 
가벼운 아침메뉴 먹으러 갔고 매장도 쾌적하고 재료들도 신선해서 맛있었어요
쿠키도 쫀득하고 시켰던 메뉴들과 조합이 생각보다 좋더라구요 / 23.11.26.일 / 1번째 방문 / 매장이 넓어요
Favor10 / 햄버거땡길때..전 써브웨이로! 키오스크 밀리지않으면 속도도 빠르고 편해용! / 23.12.13.수 / 3번째 방문 / 음식이 맛있어요
맛집학과 수석 / 섭웨는 매일 먹어도 안질려요 :) / 1.26.금 / 12번째 방문 / 매장이 넓어요
맛집학과 수석 / 간편하게 혼밥하기 더할나위 없이 좋아요 :) / 23.11.9.목 / 5번째 방문 / 음식이 맛있어요
sae**** / 주문 방식이 키오스크여서 당황했어요 
그래서 써브웨이의 특유한 분위기가 사라진 것ㅁ같아 아쉽네요.  / 1.29.월 / 1번째 방문 / 음식이 맛있어요
쪼꼬미쪼콩 